In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from sklearn.preprocessing import OneHotEncoder


# Creation of user_df, a DataFrame with informations about Archived Users

users = []

for filepath in glob.iglob('Users/Archived users/*.txt'):
    users.append(filepath[-14:-4])

# print(users)
user_number = len(users)    # number of users

user_df = pd.DataFrame(users, columns = ["Users"])

# Create columns

col_index = []

file = open('Users/Archived users/User_' + str(users[0]) + ".txt")
lines = file.read().split("\n")
ll = len(lines)

for l in range(ll-1):
    sl = lines[l].split(": ", 1)
    col_index.append(sl[0])

# print(col_index)

# Create values

values_list = []
values = []

for i in range(user_number):
    file = open('Users/Archived users/User_' + str(users[i]) + ".txt")
    lines = file.read().split("\n")
    for l in range(ll-1):
        sl = lines[l].split(": ", 1)
        v = sl[1]
        values.append(v)
    values_list.append(values)
    values = []

# Insert values in user_df

value_df = pd.DataFrame(values_list, columns = col_index)

# print (value_df)

user_df = pd.concat([user_df, value_df], axis = 1)

# print(user_df)

# Creation of tappydata_df, a DataFrame with informations about Archived Data

tappydata = []

for filepath in glob.iglob('Data/Tappy Data/*.txt'):
    file = open(filepath)
    lines = file.read().split("\n")
    for l in range(len(lines)-1):
        x = lines[l]
        x = x.split("\t")
        tappydata.append(x)

tappydata_df = pd.DataFrame(tappydata, columns = ["Users", "Date", "Datetime", "TapPosition", "Hold", "Movement", "Latency", "Flight", "*"])

# print(tappydata_df)

all_userdata_df = pd.merge(tappydata_df, user_df, how='inner', left_on='Users', right_on='Users')
#

all_userdata_df = all_userdata_df[all_userdata_df.Impact != "Severe"]
all_userdata_df = all_userdata_df[all_userdata_df.Impact != "Medium"]

all_userdata_df = all_userdata_df[all_userdata_df.Levadopa == "False"]

# counts = all_userdata_df['Users'].value_counts()
# print(counts)


In [2]:
#counts_parkinson = all_userdata_df[all_userdata_df["Parkinsons"] == "True"]
#counts_parkinson.nunique()

# quindi ciamo 34 malati e 55 sani

counts = all_userdata_df['Users'].value_counts()
#avg = all_userdata_df.shape[0] / len(counts)
#print(avg)  # numero medio di tap per ogni utente

#counts.plot(kind = "hist")
#counts.plot(kind = "hist", range = [0, 50])

# togliere da dopo 1000 a ???
# se togliamo i 3 che fanno tanti tap (3/4/5/600 k) Che succede??? quei dati influenzano troppo?

all_userdata_df = all_userdata_df[all_userdata_df['Users'].isin(counts[counts > 1000].index)]
all_userdata_df = all_userdata_df[all_userdata_df['Users'].isin(counts[counts < 100000].index)]

#all_userdata_df.head()

# non leviamo i tap con lo spazio !!!

all_userdata_df = all_userdata_df.drop(["Users", "Date", "Datetime", "Flight", "*", "Tremors", "DiagnosisYear", "Sided", "UPDRS", "Impact", "Levadopa", "DA", "MAOB", "Other"], axis=1)

# completata la pulizia



In [3]:
#print(len(all_userdata_df.loc[158453]))



# counts_Rhold = all_userdata_df[all_userdata_df["Latency"] == 'SOP2REQFUH']
# print(counts_Rhold)
# counts_Rhold.head()
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='01CAXCJIKLYM'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='EF9XEFXPBN'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='01IDYIDJBAXF'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='K2VMDST8HC'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='LSQWWDXEYO'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='04LSQWWDXEYO'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='0NMMGWRY6SO'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='08:27:14.043'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='01QCFNDCZPQH'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='09:11:16.406'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='S9C3TTX3NE'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='SOP2REQFUH'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='05SOP2REQFUH'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='0613.SOP2REQFUH'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='04TL2XHTLK1T'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='12:12:50.477'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='0218.VSBEPFSFWK'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='21:38:52.695'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Latency']=='07YQSGN9BMVK'].index, inplace=True)
all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['BirthYear']==''].index, inplace=True)



In [4]:
# print(all_userdata_df.index[719771])
# sum = 0
# for i in all_userdata_df.index:
#     for j in range(0, all_userdata_df.shape[1]):
#         sum += len(all_userdata_df.iloc[i, j])
#     print (sum)
# sum = 0
# for j in range(0, all_userdata_df.shape[1]):
#     sum += len(all_userdata_df.iloc[719771, j])
# print (sum)


#7006976


# counts_by = all_userdata_df[all_userdata_df["UPDRS"] == '11:47:33.844	']
# print(counts_by)
# counts_by.head()

# for i in all_userdata_df.index:
#         print iloc[all_userdata_df.index(1), j]

# all_userdata_df.apply(lambda row: row.astype(str).str.contains('11:47:33.844	').any(), axis=1)

In [5]:
#all_userdata_df.head()

# le colonne ora sono: tap position (L/R), Hold, Movement, BirthYear, Gender, Parkinsons (T/F)
# rimuovo gli utenti senza data di nascita, perche mi serve per calcolare l'eta

#all_userdata_df = all_userdata_df[(all_userdata_df.BirthYear != "")]

#all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Hold']=='R'].index, inplace=True)



# normalizzazione
# L/R - Gender - Parkinson : devono diventare campi binari del tipo 0/1
# movement : lo codifichiamo con onehot dividendolo in 4 colonne
# latency - hold : sono dei float
# age (ottenuta da 2017 - birthyear) : int

#all_userdata_df = all_userdata_df.rename(columns = {"TapPosition" : "RightTap"}) ##### perche ci sta anche S

#all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Hold']=='R'].index, inplace=True)
#all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Hold']=='16:15:50.375'].index, inplace=True)

# RICONTROLLARE PERCHE ESCE LA R SULLA COLONNA HOLD (RIGA 409197)

from sklearn.preprocessing import OneHotEncoder
all_userdata_df.TapPosition.replace(['R', 'L', 'S'], [0, 1, 2], inplace = True)
# va fatto il onehot

all_userdata_df.Movement.replace(['LL', 'LR', 'RR', 'RL', 'LS', 'SL', 'RS', 'SR', 'SS'], [0, 1, 2, 3, 4, 5, 6, 7, 8], inplace = True)
# va fatto il onehot

all_userdata_df.Hold = all_userdata_df.Hold.astype(float) # se ci sono problemi e per colpa dei file .txt

# TOGLIERE COMMENTO E CAPIRE PERCHE NON FUNZIONA (CANT CONVERT STRING TO FLOAT)
#counts_ss = all_userdata_df[all_userdata_df["Movement"] == "SS"]

all_userdata_df.Latency = all_userdata_df.Latency.astype(float) # se ci sono problemi e per colpa dei file .txt
# TOGLIERE COMMENTO E CAPIRE PERCHE NON FUNZIONA (CANT CONVERT STRING TO FLOAT)

all_userdata_df.BirthYear = all_userdata_df.BirthYear.astype(int)
all_userdata_df.BirthYear = 2017 - all_userdata_df.BirthYear
all_userdata_df = all_userdata_df.rename(columns = {"BirthYear" : "Age"}) # perche adesso e espresso in termini di eta

all_userdata_df.Gender.replace(['Male', 'Female'], [1, 0], inplace=True)

all_userdata_df.Parkinsons.replace(['True', 'False'], [1, 0], inplace=True)

all_userdata_df.head()

,TapPosition,Hold,Movement,Latency,Age,Gender,Parkinsons
158453,1,78.1,0,312.5,58,0,0
158454,1,78.1,0,453.1,58,0,0
158455,1,62.5,0,414.1,58,0,0
158456,1,93.8,0,742.2,58,0,0
158457,0,101.6,1,523.4,58,0,0


In [6]:
# onehot di tap position e movement

OneHotTapPosition = pd.get_dummies(all_userdata_df.TapPosition)
OneHotMovement = pd.get_dummies(all_userdata_df.Movement)

OneHotMovement = OneHotMovement.rename(columns = {0: "LL", 1: "LR", 2: "RR", 3: "RL", 4: "LS", 5: "SL", 6: "RS", 7: "SR", 8: "SS"})
all_userdata_df = all_userdata_df.drop(["Movement"], axis=1)

OneHotTapPosition = OneHotTapPosition.rename(columns = {0: "TapR", 1: "TapL", 2: "TapS"})
all_userdata_df = all_userdata_df.drop(["TapPosition"], axis=1)

all_userdata_df = pd.concat([all_userdata_df, OneHotTapPosition, OneHotMovement], axis=1)



In [7]:

#print(all_userdata_df)

# for j in all_userdata_df:
#     print(j)

all_userdata_df = all_userdata_df.reindex(columns = ["Age", "Hold", "Latency", "Gender", "TapR", "TapL", "LL", "LR", "RR", "RL", "LS", "SL", "RS", "SR", "SS", "Parkinsons"])
all_userdata_df.head(20)



,Age,Hold,Latency,Gender,TapR,TapL,LL,LR,RR,RL,LS,SL,RS,SR,SS,Parkinsons
158453,58,78.1,312.5,0,0,1,1,0,0,0,0,0,0,0,0,0
158454,58,78.1,453.1,0,0,1,1,0,0,0,0,0,0,0,0,0
158455,58,62.5,414.1,0,0,1,1,0,0,0,0,0,0,0,0,0
158456,58,93.8,742.2,0,0,1,1,0,0,0,0,0,0,0,0,0
158457,58,101.6,523.4,0,1,0,0,1,0,0,0,0,0,0,0,0
158458,58,93.8,203.1,0,1,0,0,0,1,0,0,0,0,0,0,0
158459,58,109.4,289.1,0,0,1,0,0,0,1,0,0,0,0,0,0
158460,58,214.8,316.4,0,1,0,0,1,0,0,0,0,0,0,0,0
158461,58,66.4,382.8,0,1,0,0,0,1,0,0,0,0,0,0,0
158462,58,93.8,562.5,0,0,1,0,0,0,1,0,0,0,0,0,0


In [8]:
x = all_userdata_df.iloc[:, :-1].values
y = all_userdata_df.iloc[:, -1].values

print(x)
print(y)

# ho diviso tutte le colonne (x) dal target parkinsons (y)

[[ 58.   78.1 312.5 ...   0.    0.    0. ]
 [ 58.   78.1 453.1 ...   0.    0.    0. ]
 [ 58.   62.5 414.1 ...   0.    0.    0. ]
 ...
 [ 70.  171.9 343.8 ...   0.    0.    0. ]
 [ 70.  132.8 312.5 ...   0.    0.    0. ]
 [ 70.  109.4 437.5 ...   0.    0.    0. ]]
[0 0 0 ... 1 1 1]


!!! Da qui in poi è solo copiato dal signore di Genova quindi va un po' reinventato

In [9]:
from sklearn.model_selection import train_test_split

# splitto matrice x e array y in sottoinsiemi random di train e test, di dimensioni 4/5 e 1/5
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

graph = all_userdata_df.sample(n = 100)
x_graph = graph.iloc[:, 1:3]
y_graph = graph.iloc[:, -1]

# print(x_train)
# print(y_train)

# print(x_test)
# print(y_test)

# print(x_graph)
# print(y_graph)

# prima del modello di classificazione faccio lo scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_graph = scaler.fit_transform(x_graph)
#proviamo con questo scaler dopo possiamo provare quello normaliza

print(x_graph)


[[-0.62940125 -1.21185337]
 [ 0.32099609  0.93637258]
 [-0.15298723 -0.38963414]
 [-0.05819057 -0.92058   ]
 [ 1.17416609 -0.84114181]
 [ 1.6505801   1.73211241]
 [ 0.13140276  1.41368069]
 [-0.72419791 -0.4167925 ]
 [-0.2477839   0.14063276]
 [ 0.22619943 -0.86762121]
 [-0.43980792  2.66024922]
 [ 0.70018276 -0.60214828]
 [ 1.74537677 -1.60972328]
 [ 0.13140276  0.03471517]
 [ 0.0366061   0.85625543]
 [ 0.88977609  2.97800199]
 [-2.14857858 -0.81398345]
 [-1.00858791 -0.70806587]
 [-0.62940125 -0.38963414]
 [ 2.31415676  0.56498207]
 [-0.2477839   1.99690638]
 [-0.72419791  2.15646172]
 [-0.62940125 -0.4167925 ]
 [-0.62940125 -0.38963414]
 [-0.62940125 -0.49623069]
 [-1.00858791 -0.38963414]
 [ 0.80713284 -0.14452998]
 [ 0.60538609 -0.65510707]
 [-0.05819057 -1.4508469 ]
 [ 0.32099609 -0.97353879]
 [ 1.9349701   2.15646172]
 [-0.43980792  1.01581078]
 [ 0.22619943  0.16711215]
 [-0.2477839   2.63376983]
 [ 2.02976676 -0.92058   ]
 [-1.76696124 -0.29729573]
 [-0.53460458 -1.02649759]
 

In [30]:
len(x_train)

# qua si parte con il learning
# pero il ragazzo di Genova dice che x_train e x_test hanno troppe righe
# allora si riduce le dimensioni del dataset prendendo il 4% (applica test_size = 0.2 per due volte)

#############

from sklearn.linear_model import LogisticRegression

classifierLogReg = LogisticRegression(random_state = 0)
classifierLogReg.fit(x_train, y_train)

#The algorithm has learnt on training set, now I will predict the outcomes on test set
y_pred_LogReg0 = classifierLogReg.predict(x_train)
y_pred_LogReg = classifierLogReg.predict(x_test)




In [11]:
# Logistic Regression per model fitting

# fitta il modello e poi predice gli outcome sul test set
# poi confronta i risultati in qualche modo (vedere questo)

In [12]:
# KNN to fit algorithm

In [13]:
# SVM to fit algorithm

In [14]:
# Naive Bayes classification

In [15]:
# Decision Tree classification

In [16]:
# Random Forest classification

In [17]:
# altri metodi : chi piu ne ha piu ne metta

In [18]:
# Ensemble voting

In [19]:
##### Parte dedicata alla visualizatione (plot, grafici ecc)